# Transformer training (chantier en cours)

Imports

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [2]:
from statapp.common.preprocessing import load_data, encode_data, split_into_X_y

Preprocessing maison bien moche pour le moment... ^^ (essai avec un encodage sur les mots) Les données sont placées directement dans le dossier du notebook

In [3]:
text = load_data("data/fr.train.top1M.txt", sample=0.00001)

tokens = nltk.word_tokenize(text[:10])

In [4]:
len(tokens)

5

In [5]:
vocab = list(set(tokens))

vocab_size = len(vocab)

vocab_numbers = dict(zip(vocab, range(0,len(vocab))))
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

tokens_numbers_sequences = np.array([ tokens_numbers[i:i+max_length] for i in range(len(tokens_numbers)-max_length+1)])
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

nb_sequences =  tokens_numbers_sequences.shape[0]

In [6]:
vocab_size

5

In [7]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.01)

In [8]:
print(tokens_numbers)

[2 1 0 3 4]


In [9]:
print(tokens_numbers_sequences)

tensor([[2, 1],
        [1, 0],
        [0, 3],
        [3, 4]])


In [10]:
def train_model(nb_epochs, batch_size):
    
    #What is this ?? I don't remember
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for i, batch_indices in enumerate(batchs_indices):
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            print(output[:,:-1].reshape(-1, vocab_size))
            print(batch[:,1:].flatten())
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()
            
            #Il faudrait adapter les affichages en fonction du nombre de batchs total
            running_loss += loss.item()
            #if i % 3 == 3:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1))
            running_loss = 0.

Pourquoi la cross entropy n'est pas nulle ci-dessous ??

In [26]:
nn.CrossEntropyLoss()(torch.tensor([[1.,0,0]]),torch.tensor([0]))

tensor(0.5514)

Test d'overfitting sur un cas ultrasimplifié ;
- En observant les sorties le modèle a bien appris et overfitte, mais la loss affichée ne descend pas jusque zéro !

In [13]:
train_model(50,1)

tensor([[1.2144e-16, 2.7084e-10, 4.7899e-15, 1.0000e+00, 4.0311e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[1,     1] loss: 0.905
tensor([[1.6835e-10, 1.0000e+00, 3.3588e-10, 4.2783e-13, 2.8832e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[1,     2] loss: 0.905
tensor([[9.0431e-17, 4.7918e-14, 2.5579e-13, 1.3128e-12, 1.0000e+00]],
       grad_fn=<AsStridedBackward>)
tensor([4])
[1,     3] loss: 0.905
tensor([[1.0000e+00, 9.4468e-09, 1.4817e-09, 4.1633e-13, 6.8384e-09]],
       grad_fn=<AsStridedBackward>)
tensor([0])
[1,     4] loss: 0.905
tensor([[1.0000e+00, 9.4375e-09, 1.4812e-09, 4.1618e-13, 6.8343e-09]],
       grad_fn=<AsStridedBackward>)
tensor([0])
[2,     1] loss: 0.905
tensor([[1.2119e-16, 2.6987e-10, 4.7814e-15, 1.0000e+00, 4.0082e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[2,     2] loss: 0.905
tensor([[1.6920e-10, 1.0000e+00, 3.3709e-10, 4.2948e-13, 2.8867e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[2,     3] loss: 0.905
tensor

       grad_fn=<AsStridedBackward>)
tensor([4])
[15,     2] loss: 0.905
tensor([[1.2155e-16, 2.6457e-10, 4.7841e-15, 1.0000e+00, 3.8881e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[15,     3] loss: 0.905
tensor([[1.7931e-10, 1.0000e+00, 3.5220e-10, 4.5256e-13, 2.9566e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[15,     4] loss: 0.905
tensor([[1.0000e+00, 9.0327e-09, 1.4680e-09, 4.1436e-13, 6.7106e-09]],
       grad_fn=<AsStridedBackward>)
tensor([0])
[16,     1] loss: 0.905
tensor([[1.2160e-16, 2.6430e-10, 4.7851e-15, 1.0000e+00, 3.8822e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[16,     2] loss: 0.905
tensor([[1.8004e-10, 1.0000e+00, 3.5329e-10, 4.5424e-13, 2.9617e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[16,     3] loss: 0.905
tensor([[9.0672e-17, 4.7666e-14, 2.5649e-13, 1.3166e-12, 1.0000e+00]],
       grad_fn=<AsStridedBackward>)
tensor([4])
[16,     4] loss: 0.905
tensor([[1.0000e+00, 9.0018e-09, 1.4671e-09, 4.1430e-13, 6.7012e-09]],

tensor([4])
[29,     3] loss: 0.905
tensor([[1.9215e-10, 1.0000e+00, 3.7089e-10, 4.8228e-13, 3.0425e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[29,     4] loss: 0.905
tensor([[1.0000e+00, 8.6383e-09, 1.4548e-09, 4.1349e-13, 6.5765e-09]],
       grad_fn=<AsStridedBackward>)
tensor([0])
[30,     1] loss: 0.905
tensor([[1.9267e-10, 1.0000e+00, 3.7163e-10, 4.8348e-13, 3.0457e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[30,     2] loss: 0.905
tensor([[9.1213e-17, 4.7564e-14, 2.5765e-13, 1.3230e-12, 1.0000e+00]],
       grad_fn=<AsStridedBackward>)
tensor([4])
[30,     3] loss: 0.905
tensor([[1.2256e-16, 2.5870e-10, 4.8032e-15, 1.0000e+00, 3.7608e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[30,     4] loss: 0.905
tensor([[1.2258e-16, 2.5859e-10, 4.8035e-15, 1.0000e+00, 3.7585e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[31,     1] loss: 0.905
tensor([[9.1244e-17, 4.7560e-14, 2.5771e-13, 1.3234e-12, 1.0000e+00]],
       grad_fn=<AsStridedBackward>)

       grad_fn=<AsStridedBackward>)
tensor([4])
[44,     1] loss: 0.905
tensor([[2.0681e-10, 1.0000e+00, 3.9128e-10, 5.1605e-13, 3.1302e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[44,     2] loss: 0.905
tensor([[1.0000e+00, 8.2484e-09, 1.4383e-09, 4.1226e-13, 6.4055e-09]],
       grad_fn=<AsStridedBackward>)
tensor([0])
[44,     3] loss: 0.905
tensor([[1.2352e-16, 2.5273e-10, 4.8190e-15, 1.0000e+00, 3.6347e-10]],
       grad_fn=<AsStridedBackward>)
tensor([3])
[44,     4] loss: 0.905
tensor([[9.1881e-17, 4.7493e-14, 2.5905e-13, 1.3310e-12, 1.0000e+00]],
       grad_fn=<AsStridedBackward>)
tensor([4])
[45,     1] loss: 0.905
tensor([[2.0791e-10, 1.0000e+00, 3.9277e-10, 5.1857e-13, 3.1364e-11]],
       grad_fn=<AsStridedBackward>)
tensor([1])
[45,     2] loss: 0.905
tensor([[1.0000e+00, 8.2213e-09, 1.4370e-09, 4.1216e-13, 6.3921e-09]],
       grad_fn=<AsStridedBackward>)
tensor([0])
[45,     3] loss: 0.905
tensor([[1.2358e-16, 2.5234e-10, 4.8199e-15, 1.0000e+00, 3.6266e-10]],